In [1]:
import os
import utils.cv_reviewer_utils as cv_utils


In [2]:
#if its published somewhere
#job_description = get_job_description("url")

job_description = """

Job Title: Junior Data Engineer
Location: Remote
Job Type: Full-time
Team: Data Engineering
Reports to: Senior Data Engineer

About the Role:

We are looking for a Junior Data Engineer to join our growing data team. You will support the design, development, and maintenance of data pipelines and infrastructure that power data analytics, reporting, and data-driven decision-making across the company.
This is a great opportunity for someone early in their career to learn and grow while working on real-world data challenges in a collaborative and agile environment.

Key Responsibilities
- Assist in building, maintaining, and optimizing ETL/ELT pipelines.
- Work closely with data analysts, scientists, and engineers to support data ingestion and transformation workflows.
- Help ensure the reliability and quality of data across systems.
- Monitor and troubleshoot data pipelines and processes.
- Contribute to documentation of data flows, models, and architecture.
- Learn and apply best practices in data engineering, including security and scalability.

Requirements:

- Basic knowledge of SQL and at least one programming language (Python preferred).
- Familiarity with data storage systems (e.g., relational databases, cloud storage).
- Understanding of data processing concepts and tools.
- Willingness to learn and grow in a fast-paced environment.
- Good communication and problem-solving skills.
- Bachelor’s degree in Computer Science, Engineering, Mathematics, or related field (or equivalent practical experience).

Nice to Have:

- Exposure to cloud platforms like AWS, GCP, or Azure.
- Experience with version control (e.g., Git).
- Basic knowledge of data modeling concepts.
- Familiarity with tools like Airflow, dbt, or Spark.

"""

# in lower case
mandatory_keywrods = ["spark","python"]



In [3]:
landing_path = './landing'

descriptions_dict = {}
evaluation_dict = {}
matches = []

for filename in os.listdir(landing_path):
    path = os.path.join(landing_path, filename)
    
    if os.path.isfile(path):
        cv_text = cv_utils.extract_text_from_cv(path)
        if cv_text is not None:
            
            words = cv_text.split()    
            num_of_words = len(words)

            if num_of_words > 5:    
                descriptions_dict[filename] = cv_text

for filename,candidate_desc in descriptions_dict.items():

        # "llama3.2" or "gpt-4o-mini" by now :)
        model = "gpt-4o-mini"

        # keywords string is not mandatory
        keywords_string = "Additional note: " + cv_utils.evaluate_mandatory_keywords(candidate_desc,mandatory_keywrods)
    
        llm_answer = cv_utils.evaluate_candidate(model, candidate_desc, job_description, keywords_string)
        matches.append(llm_answer)


CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox
CropBox missing from /Page, defaulting to MediaBox


▶️ Using gpt-4o-mini via OpenAI API...
▶️ Using gpt-4o-mini via OpenAI API...
▶️ Using gpt-4o-mini via OpenAI API...
▶️ Using gpt-4o-mini via OpenAI API...


In [4]:
sorted_matches = sorted(matches, key=lambda x: x['match_percentage'], reverse=True)

n = 1
for match in sorted_matches:
    print(f"candidate nº{n}: {match['name']} - {match['match_percentage']}% → {match['summary']} \n")
    n+=1


candidate nº1: Thaddeus Drake - 90% → Thaddeus has robust experience in building, maintaining, and optimizing ETL pipelines using tools like Apache NiFi and Airflow, which aligns well with the role's requirements. His proficiency in SQL and Python, along with hands-on experience in cloud services like AWS, positions him as a strong candidate. He has demonstrated skills in data quality and reliability through his work with large datasets. Although he does not mention experience with Spark or GCP/Azure, his extensive background in data engineering effectively supports the job's expectations. 

candidate nº2: ALAN SUSA - 85% → Alan has extensive experience as a Data Engineer, with skills in SQL and Python, and has worked with various data storage systems such as Redshift and S3. He has built and maintained ETL pipelines and has experience in cloud platforms like AWS. However, as he has significant work experience, he may not align with the junior level position as closely, which typically